In [ ]:
#Imports---------------------------------------------------------------
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
#%matplotlib inline
#%config InlineBackend.figure_format = 'retina'

import glob
import numpy as np
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
#Main pipeline for multiclass classification
#import libraries
import numpy as np
from time import time
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
from tqdm import tqdm
from sklearn.model_selection import KFold
from torchvision import models, transforms
import pickle
import os
import sys
from PIL import Image


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


Mounted at /content/drive


In [ ]:
K=5
arc='vgg16'
loss_func = nn.CrossEntropyLoss()
lr=0.0001
b_s=32
ep=50-5
PATH='/content/drive/My Drive/ML/model.pth'
PATH2='/content/drive/My Drive/ML/files/last.pth'
#APAGAR-------------------------------------
#with open('/content/drive/My Drive/ML/Resultados/readme.txt', 'w') as f:
    #f.write('')
#with open('/content/drive/My Drive/ML/Resultados/readme.txt', 'a') as f:
    #f.write('------------------------------------------------\nParametros:\n\nEpochs'+str(ep)+'\nArc:'+arc+'\nLearning rate:'+str(lr)+'\nBatch size:'+str(b_s))

In [ ]:
#Standerdize the image----------------------------------------------------------------------

transform = transforms.Compose([            #[1]
 #transforms.Resize(224),                    #[2]
 #transforms.CenterCrop(224),                #[3]
 transforms.ToTensor(),                     #[4]
 transforms.Normalize(                      #[5]
 mean=[0.485, 0.456, 0.406],                #[6]
 std=[0.229, 0.224, 0.225]                  #[7]
 )])

In [ ]:
infile = open('/content/drive/My Drive/ML/NCI_data.pickle','rb')
data=pickle.load(infile)
infile.close
print(data)


class MyDataset(Dataset):
   def __init__(self, type, transform, path,fold):
       self.X,self.Y=data[fold][type]
       self.transform = transform
       self.path = path
       
   def __getitem__(self, i):
       #print('.',end='')
       PATH=self.path+self.X[i]
       while(True):
         try:
           X = Image.open(PATH)
           break
         except IOError:
           print('erro'+self.X[i])
       X = self.transform(X)
       Y = self.Y[i]
       return X, Y

   def __len__(self):
       return len(self.X)


[{'train': (array(['I422042_C1.jpg', 'I422042_C2.jpg', 'I720027_C1.jpg', ...,
       'I147525_C2.jpg', 'I325700_C2.jpg', 'I553280_C2.jpg'], dtype='<U14'), array([2, 2, 2, ..., 0, 0, 0], dtype=uint8)), 'test': (array(['I673095_C1.jpg', 'I752115_C1.jpg', 'I507880_C2.jpg', ...,
       'I553280_C1.jpg', 'I853803_C1.jpg', 'I853803_C2.jpg'], dtype='<U14'), array([2, 0, 0, ..., 0, 0, 0], dtype=uint8))}, {'train': (array(['I422042_C1.jpg', 'I422042_C2.jpg', 'I720027_C1.jpg', ...,
       'I553280_C1.jpg', 'I853803_C1.jpg', 'I853803_C2.jpg'], dtype='<U14'), array([2, 2, 2, ..., 0, 0, 0], dtype=uint8)), 'test': (array(['I673095_C2.jpg', 'I892546_C1.jpg', 'I780921_C1.jpg', ...,
       'I803109_C1.jpg', 'I147525_C1.jpg', 'I553280_C2.jpg'], dtype='<U14'), array([2, 2, 0, ..., 0, 0, 0], dtype=uint8))}, {'train': (array(['I422042_C2.jpg', 'I720027_C1.jpg', 'I673095_C1.jpg', ...,
       'I553280_C2.jpg', 'I853803_C1.jpg', 'I853803_C2.jpg'], dtype='<U14'), array([2, 2, 2, ..., 0, 0, 0], dtype=uint8)), '

In [ ]:
ds= MyDataset('train',transform,'/content/drive/My Drive/ML/train/Transform/',1)
tr = DataLoader(ds, b_s, True)
#v_ds= MyDataset('val',transform,'/content/drive/My Drive/ML/CVT_1/Transform',1)
#val = DataLoader(v_ds, b_s, True)
t_ds= MyDataset('test',transform,'/content/drive/My Drive/ML/train/Transform/',1)
test = DataLoader(t_ds, b_s, True)

In [ ]:
print(len(tr))
print(len(test))

546
182


In [ ]:
# Model
class Base(nn.Module):
    def __init__(self, pretrained_model, n_outputs):
        super().__init__()
        self.n_outputs = n_outputs
        model = getattr(models, pretrained_model)(pretrained=True) #using transfer learning from a pretrained model - True
        model = nn.Sequential(*tuple(model.children())[:-1])
        last_dimension = torch.flatten(model(torch.randn(1, 3, 224, 224))).shape[0]
        self.model = nn.Sequential(
            model,
            nn.Flatten(),
            nn.Dropout(0.4),
            nn.Linear(last_dimension, 512),
            nn.Dropout(0.4),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, K)
        )

    def forward(self, x):
        return self.model(x)

    def loss(self, Yhat, Y):
        return loss_func(Yhat, Y)

    def to_proba(self, Yhat):
        return F.softmax(Yhat, 1)

    def to_classes(self, Phat):
        return Phat.argmax(1)


In [ ]:
#Give to the model the architecture and number of classes (K)
model = Base(arc, K)
#put the model on 'GPU' or 'CPU'
model = model.to(device)
#Define optimizer and scheduler 
optimizer = optim.Adam(model.parameters(),lr )

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True)


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [ ]:
running_loss=1010.0
train_acc=0.0
final_t_a=0.0
l_train=547*32
l_test=183*32
avg_acc=0.0
best_loss=1000.0
best_acc=0.0
torch.cuda.empty_cache()
best_acc_t=0
best_loss_t=1999
loss=1000

##Train the dataset
for epoch in range(ep):  # loop over the dataset multiple times
        print('\n')
        print('Iteração:',epoch)
        print('loss:',running_loss)
        print('acc:',avg_acc)
        running_loss2=0.0
        train_acc=0.0
        final_t_a=0.0
        avg_acc2=0.0
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss
            
            }, PATH2)
        with open('/content/drive/My Drive/ML/Resultados/atual.txt', 'w') as f:
                f.write('Iteração'+str(epoch))
           
        
        ##Test the dataset
        for inputs,labels in test:
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = inputs.to(device),labels.to(device)
            labels = labels.type(torch.LongTensor)
            labels=labels.to(device)

            # forward + backward + optimize
            outputs =model(inputs)
            
            
            #print(outputs)
            loss2 = loss_func(outputs, labels)
            _, predicted = torch.max(outputs, 1)
            real=labels
            correct = (predicted == real).float().sum()
            avg_acc2 +=correct*100/l_test
            print('.',end='')
           
            running_loss2 += loss2.item()
           
        print('\n')
        print('loss:',running_loss2)
        print('acc:',avg_acc2)
        print('Finished Validation')

        if avg_acc2>=best_acc and running_loss2<=best_loss :
             best_acc=avg_acc2
             best_loss=running_loss2
             torch.save(model.state_dict(), PATH)
             print('saved')
     

        running_loss = 0.0
        avg_acc = 0        
        for inputs,labels in tr:
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = inputs.to(device),labels.to(device)
            labels = labels.type(torch.LongTensor)
            labels=labels.to(device)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs =model(inputs)
            #print(outputs)
            loss = loss_func(outputs, labels)
            loss.backward()
            optimizer.step()
            
            #print(outputs)
            #print(labels)
            _, predicted = torch.max(outputs, 1)
            real=labels
            correct = (predicted == real).float().sum()
            avg_acc +=correct*100/l_train
            print('.',end='')
           
            running_loss += loss.item()
            

           

            if avg_acc>=best_acc_t and running_loss<=best_loss_t :
             best_acc_t=avg_acc
             best_loss_t=running_loss
             
            
           

print('Finished Training')


with open('/content/drive/My Drive/ML/Resultados/readme.txt', 'a') as f:
    f.write('------------------------------------------------\nResultados:\n\nLoss treino:'+str(best_loss_t)+'\nAcc treino:'+str(best_acc_t)+'\nLoss Test:'+str(best_loss)+'\nAcc test:'+str(best_acc))




Iteração: 0
loss: 1010.0
acc: 0.0
erroI813899_C2.jpg
erroI813899_C2.jpg
erroI813899_C2.jpg
erroI813899_C2.jpg
......................................................................................................................................................................................

loss: 200.50254368782043
acc: tensor(60.5533, device='cuda:0')
Finished Validation
saved
..................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

Iteração: 1
loss: 283.28978572040796
acc: tensor(79.7303, device='c

In [ ]:
#Load Model
#model = Base(, **kwargs)
#optimizer = TheOptimizerClass(*args, **kwargs)

checkpoint = torch.load(PATH2,map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()
# - or -
model.train()

Base(
  (model): Sequential(
    (0): Sequential(
      (0): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
        (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): ReLU(inplace=True)
        (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (6): ReLU(inplace=True)
        (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (8): ReLU(inplace=True)
        (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (11): ReLU(inplace=True)
        (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (13): ReLU(inplace=True)
        (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(

In [ ]:

with open('/content/drive/My Drive/ML/Resultados/readme.txt', 'a') as f:
    f.write('------------------------------------------------\nResultados:\n\nLoss treino:'+str(best_loss_t)+'\nAcc treino:'+str(best_acc_t)+'\nLoss Test:'+str(best_loss)+'\nAcc test:'+str(best_acc))


In [ ]:
name='I625976_C1'
PATH='/content/drive/My Drive/ML/ALTS_1/'
p=PATH+name
img=Image.open(p)

FileNotFoundError: ignored

In [ ]:
model = Base(arc, K)
#put the model on 'GPU' or 'CPU'
model = model.to(device)
model.load_state_dict(torch.load(PATH))
model.eval()

#evaluate model performance
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

y_pred=predict_proba(val)
Phat = []

for inputs,labels in val:
            y_true=labels
            Phat += list(y_true.cpu().numpy())


print(Phat)
print(y_pred)
print('Classification Report:\n', classification_report(Phat, y_pred))

print('Confusion Matrix:')
cm = sns.heatmap(confusion_matrix(Phat, y_pred), annot=True, cmap='Blues')
plt.show()